# Парсер html файлов

Библиотеки и подключение гугл драйв

In [64]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 29.1 MB/s eta 0:00:00


In [72]:
from bs4 import BeautifulSoup
from pathlib import Path
import os
import pandas as pd
import re
from rapidfuzz import process
from google.colab import drive

In [47]:
drive.mount('/content/drive')
directory = '/content/drive/MyDrive/PEK/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## CMK

In [48]:
cmk_directory = directory + 'СМК'

In [55]:
frames = [
    pd.DataFrame(
        [[os.path.splitext(filename)[0],
          ' '.join(
              p.get_text(strip=True) for p in BeautifulSoup(open(os.path.join(cmk_directory, filename), encoding='utf-8').read(), 'html.parser').find_all('p')
          ) + ' ' +
          ' '.join(
              td.get_text(strip=True) for td in BeautifulSoup(open(os.path.join(cmk_directory, filename), encoding='utf-8').read(), 'html.parser').find_all('td')
          )
        ]],
        columns=['Quest', 'Ans']
    )
    for filename in os.listdir(cmk_directory)
    if os.path.isfile(os.path.join(cmk_directory, filename)) and filename.endswith('.html')
]
df = pd.concat(frames, ignore_index=True)
df

,Quest,Ans
0,Внутрикорпоративные термины - Продуктовая книг...,"Доставка товара в определенные дату и время, в..."
1,Стандарт _Оформление выдачи груза_ - CMK - Баз...,АКТУАЛЬНО Статус конфиденциальности Для внутре...
2,Концепт ролевой модели прав в новом ЛК,Права полномочий пользователя ЛК (свойства лог...
3,Создание договорных документов в Фенайс,Часто задаваемые вопросы Скорее всего у вас не...
4,Программа лояльности,Регистрация в программе лояльности в офисе Ч...
...,...,...
107,Технологическая инструкция _Проверка данных ко...,АКТУАЛЬНО Статус конфиденциальности Для внутре...
108,Инструкция пользователя «Создание нового Контр...,АКТУАЛЬНО (Взамен: ИП-УКС-001 Инструкции польз...
109,_ПРОЕКТ_ Стандарт качества процесса «Приемка г...,Статус конфиденциальности Для внутреннего испо...
110,Технологическая инструкция _Архивация документ...,АКТУАЛЬНО Статус конфиденциальности Для внутре...


Очистка от ненужных фраз

In [56]:
fix_text = 'Добавить комментарий ОЦЕНОЧНАЯ ЛИЦЕНЗИЯВам нравится Confluence? Пожалуйста, рассмотритедля приобретения егосегодня.'
fix_ends = [' - CMK - База знаний', ' - База знаний']
fix_text2 = 'Для внутреннего использования Область регламентации'
fix_text3 = 'ПРОЕКТ'

In [57]:
df['Ans'] = df['Ans'].apply(
    lambda text: text.split(fix_text2, 1)[1]
    if fix_text2 in text else text
)

df['Ans'] = df['Ans'].apply(
    lambda text: text.replace(fix_text, '').strip()
)

df['Quest'] = df['Quest'].apply(
    lambda name: next((name[:-len(end)].strip() for end in fix_ends if name.endswith(end)), name)
)

df['Ans'] = df['Ans'].apply(
    lambda text: text.replace(fix_text3, '').strip()
)
df

,Quest,Ans
0,Внутрикорпоративные термины - Продуктовая книга,"Доставка товара в определенные дату и время, в..."
1,Стандарт _Оформление выдачи груза_,Устанавливает требования к процедуре оформлени...
2,Концепт ролевой модели прав в новом ЛК,Права полномочий пользователя ЛК (свойства лог...
3,Создание договорных документов в Фенайс,Часто задаваемые вопросы Скорее всего у вас не...
4,Программа лояльности,Регистрация в программе лояльности в офисе Ча...
...,...,...
107,Технологическая инструкция _Проверка данных ко...,Требования к описанию операции «Проверка данны...
108,Инструкция пользователя «Создание нового Контр...,Требования к описанию операции «Создание новог...
109,_ПРОЕКТ_ Стандарт качества процесса «Приемка г...,Требования к процессу приемки груза на склад О...
110,Технологическая инструкция _Архивация документ...,Установление единых правил архивации документо...


Считывание ссылок

In [58]:
links = pd.read_csv(directory + 'CMK.txt', sep='/t', header=None)
links[['Title', 'URL']] = links[0].str.split(';', n=1, expand=True)
links

<ipython-input-58-0a7b03c90eac>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  links = pd.read_csv(directory + 'CMK.txt', sep='/t', header=None)


,0,Title,URL
0,"Стандарт ""Оформление выдачи груза"";http://conf...","Стандарт ""Оформление выдачи груза""",http://confluence/pages/viewpage.action?pageId...
1,"Технологическая инструкция ""Визуальная проверк...","Технологическая инструкция ""Визуальная проверк...",http://confluence/pages/viewpage.action?pageId...
2,"Технологическая инструкция ""Проверка данных ко...","Технологическая инструкция ""Проверка данных ко...",http://confluence/pages/viewpage.action?pageId...
3,Технологическая инструкция «Правила подписания...,Технологическая инструкция «Правила подписания...,http://confluence/pages/viewpage.action?pageId...
4,"Технологическая инструкция ""Архивация документ...","Технологическая инструкция ""Архивация документ...",http://confluence/pages/viewpage.action?pageId...
...,...,...,...
107,GLOBAL;http://confluence/display/prod/GLOBAL,GLOBAL,http://confluence/display/prod/GLOBAL
108,Внутрикорпоративные термины;http://confluence/...,Внутрикорпоративные термины,http://confluence/pages/viewpage.action?pageId...
109,Концепт ролевой модели прав в новом ЛК;http://...,Концепт ролевой модели прав в новом ЛК,http://confluence/pages/viewpage.action?pageId...
110,Программа лояльности;http://confluence/pages/v...,Программа лояльности,http://confluence/pages/viewpage.action?pageId...


Функция предобработки знаков препинания

In [59]:
def remove_punctuation(df, columns):
    pattern = r'[^\w\s]'

    for col in columns:
        if col in df.columns:
            df[col] = df[col].astype(str).apply(lambda x: re.sub(pattern, '', x))

    return df


links = remove_punctuation(links, ['Title'])
df = remove_punctuation(df, ['Ans', 'Quest'])
df

,Quest,Ans
0,Внутрикорпоративные термины Продуктовая книга,Доставка товара в определенные дату и время в ...
1,Стандарт _Оформление выдачи груза_,Устанавливает требования к процедуре оформлени...
2,Концепт ролевои модели прав в новом ЛК,Права полномочий пользователя ЛК свойства логи...
3,Создание договорных документов в Фенаис,Часто задаваемые вопросы Скорее всего у вас не...
4,Программа лояльности,Регистрация в программе лояльности в офисе Ча...
...,...,...
107,Технологическая инструкция _Проверка данных ко...,Требования к описанию операции Проверка данных...
108,Инструкция пользователя Создание нового Контра...,Требования к описанию операции Создание нового...
109,_ПРОЕКТ_ Стандарт качества процесса Приемка гр...,Требования к процессу приемки груза на склад О...
110,Технологическая инструкция _Архивация документ...,Установление единых правил архивации документо...


In [60]:
links

,0,Title,URL
0,"Стандарт ""Оформление выдачи груза"";http://conf...",Стандарт Оформление выдачи груза,http://confluence/pages/viewpage.action?pageId...
1,"Технологическая инструкция ""Визуальная проверк...",Технологическая инструкция Визуальная проверка...,http://confluence/pages/viewpage.action?pageId...
2,"Технологическая инструкция ""Проверка данных ко...",Технологическая инструкция Проверка данных кон...,http://confluence/pages/viewpage.action?pageId...
3,Технологическая инструкция «Правила подписания...,Технологическая инструкция Правила подписания ...,http://confluence/pages/viewpage.action?pageId...
4,"Технологическая инструкция ""Архивация документ...",Технологическая инструкция Архивация документо...,http://confluence/pages/viewpage.action?pageId...
...,...,...,...
107,GLOBAL;http://confluence/display/prod/GLOBAL,GLOBAL,http://confluence/display/prod/GLOBAL
108,Внутрикорпоративные термины;http://confluence/...,Внутрикорпоративные термины,http://confluence/pages/viewpage.action?pageId...
109,Концепт ролевой модели прав в новом ЛК;http://...,Концепт ролевой модели прав в новом ЛК,http://confluence/pages/viewpage.action?pageId...
110,Программа лояльности;http://confluence/pages/v...,Программа лояльности,http://confluence/pages/viewpage.action?pageId...


Объединение таблиц

In [67]:
links['Sovpad'] = links['Title'].apply(
    lambda x: (process.extractOne(x, df['Quest'], score_cutoff=75) or (None,))[0]
)
links

,0,Title,URL,Sovpad
0,"Стандарт ""Оформление выдачи груза"";http://conf...",Стандарт Оформление выдачи груза,http://confluence/pages/viewpage.action?pageId...,Стандарт _Оформление выдачи груза_
1,"Технологическая инструкция ""Визуальная проверк...",Технологическая инструкция Визуальная проверка...,http://confluence/pages/viewpage.action?pageId...,Технологическая инструкция _Визуальная проверк...
2,"Технологическая инструкция ""Проверка данных ко...",Технологическая инструкция Проверка данных кон...,http://confluence/pages/viewpage.action?pageId...,Технологическая инструкция _Проверка данных ко...
3,Технологическая инструкция «Правила подписания...,Технологическая инструкция Правила подписания ...,http://confluence/pages/viewpage.action?pageId...,Технологическая инструкция Правила подписания ...
4,"Технологическая инструкция ""Архивация документ...",Технологическая инструкция Архивация документо...,http://confluence/pages/viewpage.action?pageId...,Технологическая инструкция _Архивация документ...
...,...,...,...,...
107,GLOBAL;http://confluence/display/prod/GLOBAL,GLOBAL,http://confluence/display/prod/GLOBAL,GLOBAL Продуктовая книга
108,Внутрикорпоративные термины;http://confluence/...,Внутрикорпоративные термины,http://confluence/pages/viewpage.action?pageId...,Внутрикорпоративные термины Продуктовая книга
109,Концепт ролевой модели прав в новом ЛК;http://...,Концепт ролевой модели прав в новом ЛК,http://confluence/pages/viewpage.action?pageId...,Концепт ролевои модели прав в новом ЛК
110,Программа лояльности;http://confluence/pages/v...,Программа лояльности,http://confluence/pages/viewpage.action?pageId...,Программа лояльности


In [68]:
best_match_to_link = dict(zip(links['Sovpad'], links['URL']))

In [69]:
def find_link(query):
    for best_match, link in best_match_to_link.items():
        if best_match is not None and query.startswith(best_match):
            return link
    return None

df['URL'] = df['Quest'].apply(find_link)
df

,Quest,Ans,URL
0,Внутрикорпоративные термины Продуктовая книга,Доставка товара в определенные дату и время в ...,http://confluence/pages/viewpage.action?pageId...
1,Стандарт _Оформление выдачи груза_,Устанавливает требования к процедуре оформлени...,http://confluence/pages/viewpage.action?pageId...
2,Концепт ролевои модели прав в новом ЛК,Права полномочий пользователя ЛК свойства логи...,http://confluence/pages/viewpage.action?pageId...
3,Создание договорных документов в Фенаис,Часто задаваемые вопросы Скорее всего у вас не...,http://confluence/pages/viewpage.action?pageId...
4,Программа лояльности,Регистрация в программе лояльности в офисе Ча...,http://confluence/pages/viewpage.action?pageId...
...,...,...,...
107,Технологическая инструкция _Проверка данных ко...,Требования к описанию операции Проверка данных...,http://confluence/pages/viewpage.action?pageId...
108,Инструкция пользователя Создание нового Контра...,Требования к описанию операции Создание нового...,http://confluence/pages/viewpage.action?pageId...
109,_ПРОЕКТ_ Стандарт качества процесса Приемка гр...,Требования к процессу приемки груза на склад О...,http://confluence/pages/viewpage.action?pageId...
110,Технологическая инструкция _Архивация документ...,Установление единых правил архивации документо...,http://confluence/pages/viewpage.action?pageId...


In [71]:
df['Department'] = 'CMK'
df.to_csv('df_cmk.csv', index=False)

Аналогично сделаем с другими департаментами


## DRK

In [80]:
directory_drk = directory + 'DRK/'

def parse_html(file_path, parse_tables=True):
    with open(file_path, 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f, 'html.parser')

    base_name = file_path.stem

    if parse_tables:
        return [
            pd.read_html(str(table))[0].assign(Query=base_name)
            for table in soup.find_all('table')
        ]
    else:
        paragraphs_text = ' '.join(p.get_text(strip=True) for p in soup.find_all('p'))
        return [pd.DataFrame([[base_name, paragraphs_text]], columns=['Query', 'Answer'])]

def process_directory(directory):
    p = Path(directory)

    dfs = [
        df
        for file_path in p.glob('*.html')
        for df in parse_html(
            file_path,
            parse_tables=('вопрос' in file_path.name.lower() and 'задава' not in file_path.name.lower())
        )
    ]

    tables_dfs = [df for df in dfs if set(['Query']).issubset(df.columns) and all(isinstance(c, int) for c in df.columns if c != 'Query')]
    paragraphs_dfs = [df for df in dfs if set(['Query', 'Answer']).issubset(df.columns)]
    combined_tables_df = pd.concat(tables_dfs, ignore_index=True) if tables_dfs else pd.DataFrame()
    combined_paragraphs_df = pd.concat(paragraphs_dfs, ignore_index=True) if paragraphs_dfs else pd.DataFrame()


    if not combined_tables_df.empty:
        combined_tables_df = combined_tables_df.drop(index=0, errors='ignore')
        combined_tables_df['Answer'] = combined_tables_df[0].astype(str) + ' ' + combined_tables_df[1].astype(str)
        combined_tables_df = combined_tables_df.drop([0, 1], axis=1, errors='ignore')

    return combined_tables_df, combined_paragraphs_df

table_1, table_2 = process_directory(directory_drk)
drk_df = pd.concat([table_1, table_2], axis=0).reset_index(drop=True)
drk_df

<ipython-input-80-14ad57f3b03f>:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(str(table))[0].assign(Query=base_name)
<ipython-input-80-14ad57f3b03f>:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(str(table))[0].assign(Query=base_name)
<ipython-input-80-14ad57f3b03f>:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(str(table))[0].assign(Query=base_name)


,Query,Answer
0,Вопросы по личному кабинету - База знаний ДРК...,Как подключить ЛК? 1. Зайти на сайт pecom.ru2....
1,Вопросы по личному кабинету - База знаний ДРК...,"Я не хочу отправлять вам скан своего паспорта,..."
2,Вопросы по личному кабинету - База знаний ДРК...,"Я могу зарегистрироваться в ЛК, как физическое..."
3,Вопросы по личному кабинету - База знаний ДРК...,"У меня есть ЛК для организации, мне нужно доба..."
4,Вопросы по личному кабинету - База знаний ДРК...,"У меня несколько юр. лиц, я хочу сделать личны..."
...,...,...
372,Клиент не согласен с условиями оказания услуг ...,Осуществлять перевозки через Компанию «ПЭК» мо...
373,Неоформленные грузы,"Проверьте наличие заявки на забор груза, незав..."
374,Обращения в СФИНКС,СФИНКС – система фиксации и накопления клиент...
375,Частичная выдача груза,Если груз прибыл частично возможна выдача част...


Обработка

In [83]:
fix_ends = [' - База знаний ДРК', ' - База знаний']
drk_df['Query'] = drk_df['Query'].apply(
    lambda name: next((name[:-len(end)].strip() for end in fix_ends if name.endswith(end)), name)
)
drk_df['Answer'] = drk_df['Answer'].apply(
    lambda text: text.replace(fix_text, '').strip()
)

drk_df = drk_df.rename(columns={'Query': 'Quest', 'Answer': 'Ans'})
drk_df

,Quest,Ans
0,Вопросы по личному кабинету,Как подключить ЛК? 1. Зайти на сайт pecom.ru2....
1,Вопросы по личному кабинету,"Я не хочу отправлять вам скан своего паспорта,..."
2,Вопросы по личному кабинету,"Я могу зарегистрироваться в ЛК, как физическое..."
3,Вопросы по личному кабинету,"У меня есть ЛК для организации, мне нужно доба..."
4,Вопросы по личному кабинету,"У меня несколько юр. лиц, я хочу сделать личны..."
...,...,...
372,Клиент не согласен с условиями оказания услуг ...,Осуществлять перевозки через Компанию «ПЭК» мо...
373,Неоформленные грузы,"Проверьте наличие заявки на забор груза, незав..."
374,Обращения в СФИНКС,СФИНКС – система фиксации и накопления клиентс...
375,Частичная выдача груза,Если груз прибыл частично возможна выдача част...


In [84]:
links = pd.read_csv(directory + 'DRK.txt', sep='/t', header=None)
links

<ipython-input-84-717d7ecf7d3c>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  links = pd.read_csv(directory + 'DRK.txt', sep='/t', header=None)


,0
0,Топливная надбавка_топливный сбор в ПЭ_ЭР;http...
1,Неоформленные грузы;http://confluence/pages/vi...
2,Выдача грузов по СМС при доставке;http://confl...
3,Клиент просит отключить уведомления (E-mail; В...
4,Интернет-Эквайринг (оплата банковской картой н...
...,...
295,СКРИПТЫ разговоров с клиентами;http://confluen...
296,Консультация по грузам с забором/доставкой;htt...
297,Прием/выдача грузов в ПВЗ (пункт выдачи заказо...
298,Нештатные ситуации при доставке грузов на ПВЗ;...


In [85]:
links[['Title', 'URL']] = links[0].str.split(';', n=1, expand=True)
links

,0,Title,URL
0,Топливная надбавка_топливный сбор в ПЭ_ЭР;http...,Топливная надбавка_топливный сбор в ПЭ_ЭР,http://confluence/pages/viewpage.action?pageId...
1,Неоформленные грузы;http://confluence/pages/vi...,Неоформленные грузы,http://confluence/pages/viewpage.action?pageId...
2,Выдача грузов по СМС при доставке;http://confl...,Выдача грузов по СМС при доставке,http://confluence/pages/viewpage.action?pageId...
3,Клиент просит отключить уведомления (E-mail; В...,Клиент просит отключить уведомления (E-mail,ВК; Viber; СМС; Автообзвон; Обзвон);http://co...
4,Интернет-Эквайринг (оплата банковской картой н...,Интернет-Эквайринг (оплата банковской картой н...,http://confluence/pages/viewpage.action?pageId...
...,...,...,...
295,СКРИПТЫ разговоров с клиентами;http://confluen...,СКРИПТЫ разговоров с клиентами,http://confluence/pages/viewpage.action?pageId...
296,Консультация по грузам с забором/доставкой;htt...,Консультация по грузам с забором/доставкой,http://confluence/pages/viewpage.action?pageId...
297,Прием/выдача грузов в ПВЗ (пункт выдачи заказо...,Прием/выдача грузов в ПВЗ (пункт выдачи заказов),http://confluence/pages/viewpage.action?pageId...
298,Нештатные ситуации при доставке грузов на ПВЗ;...,Нештатные ситуации при доставке грузов на ПВЗ,http://confluence/pages/viewpage.action?pageId...


In [86]:
links = remove_punctuation(links, ['Title'])
drk_df = remove_punctuation(drk_df, ['Ans', 'Quest'])

In [87]:
links['Sovpad'] = links['Title'].apply(
    lambda x: (process.extractOne(x, drk_df['Quest'], score_cutoff=75) or (None,))[0]
)
links

,0,Title,URL,Sovpad
0,Топливная надбавка_топливный сбор в ПЭ_ЭР;http...,Топливная надбавка_топливный сбор в ПЭ_ЭР,http://confluence/pages/viewpage.action?pageId...,Топливная надбавка_топливныи сбор в ПЭ_ЭР
1,Неоформленные грузы;http://confluence/pages/vi...,Неоформленные грузы,http://confluence/pages/viewpage.action?pageId...,Неоформленные грузы
2,Выдача грузов по СМС при доставке;http://confl...,Выдача грузов по СМС при доставке,http://confluence/pages/viewpage.action?pageId...,Выдача грузов по СМС при доставке
3,Клиент просит отключить уведомления (E-mail; В...,Клиент просит отключить уведомления Email,ВК; Viber; СМС; Автообзвон; Обзвон);http://co...,Клиент просит отключить уведомления Email_ ВК_...
4,Интернет-Эквайринг (оплата банковской картой н...,ИнтернетЭквайринг оплата банковской картой на ...,http://confluence/pages/viewpage.action?pageId...,ИнтернетЭкваиринг оплата банковскои картои на ...
...,...,...,...,...
295,СКРИПТЫ разговоров с клиентами;http://confluen...,СКРИПТЫ разговоров с клиентами,http://confluence/pages/viewpage.action?pageId...,СКРИПТЫ разговоров с клиентами
296,Консультация по грузам с забором/доставкой;htt...,Консультация по грузам с заборомдоставкой,http://confluence/pages/viewpage.action?pageId...,Консультация по грузам с забором_доставкои
297,Прием/выдача грузов в ПВЗ (пункт выдачи заказо...,Приемвыдача грузов в ПВЗ пункт выдачи заказов,http://confluence/pages/viewpage.action?pageId...,Прием_выдача грузов в ПВЗ пункт выдачи заказов
298,Нештатные ситуации при доставке грузов на ПВЗ;...,Нештатные ситуации при доставке грузов на ПВЗ,http://confluence/pages/viewpage.action?pageId...,Нештатные ситуации при доставке грузов на ПВЗ


In [88]:
best_match_to_link = dict(zip(links['Sovpad'], links['URL']))

def find_link(query):
    for best_match, link in best_match_to_link.items():
        if best_match is not None and query.startswith(best_match):
            return link
    return None

drk_df['URL'] = drk_df['Quest'].apply(find_link)

drk_df['Department'] = 'DRK'
drk_df.to_csv('df_drk.csv', index=False)

drk_df

,Quest,Ans,URL,Department
0,Вопросы по личному кабинету,Как подключить ЛК 1 Зайти на сайт pecomru2 В л...,http://confluence/pages/viewpage.action?pageId...,DRK
1,Вопросы по личному кабинету,Я не хочу отправлять вам скан своего паспорта ...,http://confluence/pages/viewpage.action?pageId...,DRK
2,Вопросы по личному кабинету,Я могу зарегистрироваться в ЛК как физическое ...,http://confluence/pages/viewpage.action?pageId...,DRK
3,Вопросы по личному кабинету,У меня есть ЛК для организации мне нужно добав...,http://confluence/pages/viewpage.action?pageId...,DRK
4,Вопросы по личному кабинету,У меня несколько юр лиц я хочу сделать личный ...,http://confluence/pages/viewpage.action?pageId...,DRK
...,...,...,...,...
372,Клиент не согласен с условиями оказания услуг ...,Осуществлять перевозки через Компанию ПЭК можн...,http://confluence/pages/viewpage.action?pageI...,DRK
373,Неоформленные грузы,Проверьте наличие заявки на забор груза незаве...,http://confluence/pages/viewpage.action?pageId...,DRK
374,Обращения в СФИНКС,СФИНКС система фиксации и накопления клиентск...,http://confluence/pages/viewpage.action?pageId...,DRK
375,Частичная выдача груза,Если груз прибыл частично возможна выдача част...,http://confluence/pages/viewpage.action?pageId...,DRK


## FT

In [89]:
directory_FT = directory + 'ФТ'

paragraphs_dataframes = [
    pd.DataFrame(
        [[
            os.path.splitext(filename)[0],
            ' '.join(
                p.get_text(strip=True) for p in BeautifulSoup(open(os.path.join(directory_FT, filename), encoding='utf-8').read(), 'html.parser')
                .find_all(['p', 'td', 'span'])
            )
        ]],
        columns=['Query', 'Answer']
    )
    for filename in os.listdir(directory_FT)
    if os.path.isfile(os.path.join(directory_FT, filename)) and filename.endswith('.html')
]

df_ft = pd.concat(paragraphs_dataframes, ignore_index=True)
df_ft

,Query,Answer
0,ФТ (EPIC-13431),Связанные приложения База знаний Пространства ...
1,ФТ (EPIC-13218),Связанные приложения База знаний Пространства ...
2,ФТ (EPIC-13330),Связанные приложения База знаний Пространства ...
3,ФТ (EPIC-13642),Связанные приложения База знаний Пространства ...
4,ФТ (EPIC-13707),Связанные приложения База знаний Пространства ...
...,...,...
194,ФТ (EPIC-7260),Связанные приложения База знаний Пространства ...
195,ФТ (EPIC-6660),Связанные приложения База знаний Пространства ...
196,ФТ (EPIC-5293),Связанные приложения База знаний Пространства ...
197,ФТ (EPIC-10146),Связанные приложения База знаний Пространства ...


In [90]:
fixtext1 = 'ОЦЕНОЧНАЯ ЛИЦЕНЗИЯВам нравится Confluence? Пожалуйста, рассмотритедля приобретения егосегодня.'
fixtext2 = 'СТАТУС'

df_ft['Answer'] = df_ft['Answer'].str.replace(fixtext1, '', regex=False)
df_ft['Answer'] = df_ft['Answer'].str.replace(fixtext2, '', regex=False)

df_ft

,Query,Answer
0,ФТ (EPIC-13431),Связанные приложения База знаний Пространства ...
1,ФТ (EPIC-13218),Связанные приложения База знаний Пространства ...
2,ФТ (EPIC-13330),Связанные приложения База знаний Пространства ...
3,ФТ (EPIC-13642),Связанные приложения База знаний Пространства ...
4,ФТ (EPIC-13707),Связанные приложения База знаний Пространства ...
...,...,...
194,ФТ (EPIC-7260),Связанные приложения База знаний Пространства ...
195,ФТ (EPIC-6660),Связанные приложения База знаний Пространства ...
196,ФТ (EPIC-5293),Связанные приложения База знаний Пространства ...
197,ФТ (EPIC-10146),Связанные приложения База знаний Пространства ...


In [92]:
links = pd.read_csv(directory + 'FT.txt', delimiter=';', header=None, names=['Tittle', 'URL'], encoding = 'cp1251')
links

,Tittle,URL
0,Копия ФТ (EPIC-11361),http://confluence/pages/viewpage.action?pageId...
1,НЕ АКТУАЛЬНОЕ! ФТ (EPIC-10992) Выбор продукта ...,http://confluence/pages/viewpage.action?pageId...
2,ФТ (EPIC 5844) Интеграция с проценщиком посред...,http://confluence/pages/viewpage.action?pageId...
3,ФТ (EPIC-10132),http://confluence/pages/viewpage.action?pageId...
4,ФТ (EPIC-10142),http://confluence/pages/viewpage.action?pageId...
...,...,...
823,ФТ Реализация алгоритма расчета по новой услуг...,http://confluence/pages/viewpage.action?pageId...
824,ФТ Создание СУ Скидка на услугу Организация ст...,http://confluence/pages/viewpage.action?pageId...
825,"ФТ СУ Консолидированный забор для компаний, вх...",http://confluence/pages/viewpage.action?pageId...
826,ФТ_2 (EPIC-11408),http://confluence/pages/viewpage.action?pageId...


In [93]:
links = remove_punctuation(links, ['Title'])
df_ft = remove_punctuation(drk_df, ['Ans', 'Quest'])

In [103]:
links['Sovpad'] = links['Tittle'].apply(
    lambda x: (process.extractOne(x, df_ft['Quest'], score_cutoff=75) or (None,))[0]
)
links

,Tittle,URL,Sovpad
0,Копия ФТ (EPIC-11361),http://confluence/pages/viewpage.action?pageId...,None
1,НЕ АКТУАЛЬНОЕ! ФТ (EPIC-10992) Выбор продукта ...,http://confluence/pages/viewpage.action?pageId...,Вопросы по грузам с наложенным платежом
2,ФТ (EPIC 5844) Интеграция с проценщиком посред...,http://confluence/pages/viewpage.action?pageId...,СКРИПТЫ разговоров с клиентами
3,ФТ (EPIC-10132),http://confluence/pages/viewpage.action?pageId...,None
4,ФТ (EPIC-10142),http://confluence/pages/viewpage.action?pageId...,None
...,...,...,...
823,ФТ Реализация алгоритма расчета по новой услуг...,http://confluence/pages/viewpage.action?pageId...,Вопросы по личному кабинету
824,ФТ Создание СУ Скидка на услугу Организация ст...,http://confluence/pages/viewpage.action?pageId...,Нештатные ситуации при доставке грузов на ПВЗ
825,"ФТ СУ Консолидированный забор для компаний, вх...",http://confluence/pages/viewpage.action?pageId...,Прием_выдача грузов в ПВЗ пункт выдачи заказов
826,ФТ_2 (EPIC-11408),http://confluence/pages/viewpage.action?pageId...,None


In [104]:
best_match_to_link = dict(zip(links['Sovpad'], links['URL']))

def find_link(query):
    for best_match, link in best_match_to_link.items():
        if best_match is not None and query.startswith(best_match):
            return link
    return None

df_ft['URL'] = df_ft['Quest'].apply(find_link)

df_ft['Department'] = 'FT'
df_ft.to_csv('df_ft.csv', index=False)

df_ft

,Quest,Ans,URL,Department
0,Вопросы по личному кабинету,Как подключить ЛК 1 Зайти на сайт pecomru2 В л...,http://confluence/pages/viewpage.action?pageId...,FT
1,Вопросы по личному кабинету,Я не хочу отправлять вам скан своего паспорта ...,http://confluence/pages/viewpage.action?pageId...,FT
2,Вопросы по личному кабинету,Я могу зарегистрироваться в ЛК как физическое ...,http://confluence/pages/viewpage.action?pageId...,FT
3,Вопросы по личному кабинету,У меня есть ЛК для организации мне нужно добав...,http://confluence/pages/viewpage.action?pageId...,FT
4,Вопросы по личному кабинету,У меня несколько юр лиц я хочу сделать личный ...,http://confluence/pages/viewpage.action?pageId...,FT
...,...,...,...,...
372,Клиент не согласен с условиями оказания услуг ...,Осуществлять перевозки через Компанию ПЭК можн...,None,FT
373,Неоформленные грузы,Проверьте наличие заявки на забор груза незаве...,None,FT
374,Обращения в СФИНКС,СФИНКС система фиксации и накопления клиентск...,None,FT
375,Частичная выдача груза,Если груз прибыл частично возможна выдача част...,None,FT


In [106]:
df_ft.to_csv('df_ft.csv', index=False)

## FTL

In [136]:
directory_ftl = directory + 'FTL'

paragraphs_dataframes = [
    pd.DataFrame(
        [[
            os.path.splitext(filename)[0],
            ' '.join(
                tag.get_text(strip=True)
                for tag in BeautifulSoup(open(os.path.join(directory_ftl, filename), encoding='utf-8').read(), 'html.parser')
                .find_all(['p', 'td'])
            )
        ]],
        columns=['Query', 'Answer']
    )
    for filename in os.listdir(directory_ftl)
    if os.path.isfile(os.path.join(directory_ftl, filename)) and filename.endswith('.html')
]

df_ftl = pd.concat(paragraphs_dataframes, ignore_index=True)
df_ftl

,Query,Answer
0,Редактирование географических зон - Full Truck...,Открыть обработку «Редактирование географичес...
1,Услуги по страхованию груза и срока при перево...,EPIC-11060-Получение подробных данных проблемы...
2,Отчет по КБ - в проде - Full Truck Load - База...,3.1.Период– это дата подачи ТС под загрузку по...
3,Отчет по агентским вознаграждениям - MVP - Ful...,"По каждому рейсу, где клиент-заказчик был прив..."
4,Калькулятор FTL - MVP (но еще не обновлена) - ...,https://evgenia-petrova.atlassian.net/browse/F...
5,Комплекты документов - MVP - Full Truck Load -...,EPIC-10380-Получение подробных данных проблемы...
6,Отчет по рейсам - MVP - Full Truck Load - Баз...,EPIC-10889-Получение подробных данных проблемы...
7,Обработка для сотрудников бухгалтерии - MVP - ...,"Обработка находится в разделе ""Сервисы"" Досту..."
8,ИС 1С_ TMS настройка приложения - в проде - F...,Целевая группа – пользователи и администратор ...
9,Профили групп доступа - в проде - Full Truck L...,Группа доступа Группа доступа Разрешенные дейс...


In [137]:
fix_text = [' - в проде', ' - Full Truck Load - База знаний']
for x in fix_text:
  df_ftl['Query'] = df_ftl['Query'].str.replace(x, '', regex=False)

df_ftl

,Query,Answer
0,Редактирование географических зон,Открыть обработку «Редактирование географичес...
1,Услуги по страхованию груза и срока при перево...,EPIC-11060-Получение подробных данных проблемы...
2,Отчет по КБ,3.1.Период– это дата подачи ТС под загрузку по...
3,Отчет по агентским вознаграждениям - MVP,"По каждому рейсу, где клиент-заказчик был прив..."
4,Калькулятор FTL - MVP (но еще не обновлена),https://evgenia-petrova.atlassian.net/browse/F...
5,Комплекты документов - MVP,EPIC-10380-Получение подробных данных проблемы...
6,Отчет по рейсам - MVP,EPIC-10889-Получение подробных данных проблемы...
7,Обработка для сотрудников бухгалтерии - MVP,"Обработка находится в разделе ""Сервисы"" Досту..."
8,ИС 1С_ TMS настройка приложения,Целевая группа – пользователи и администратор ...
9,Профили групп доступа,Группа доступа Группа доступа Разрешенные дейс...


In [140]:
links = pd.read_csv(directory + 'FTL.txt', sep='/t', header=None)
links

<ipython-input-140-8306388d3273>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  links = pd.read_csv(directory + 'FTL.txt', sep='/t', header=None)


,0
0,АРМ логиста FTL - в проде;http://confluence/pa...
1,АРМ менеджера FTL - в проде;http://confluence/...
2,Алгоритм действий сотрудников по работе с заяв...
3,Алгоритм действий сотрудников по работе с заяв...
4,Документ Рейс - MVP;http://confluence/pages/vi...
5,Загрузка адресного классификатора в ИС 1С: TMS...
6,Загрузка прайса по услугам FTL в 1С: TMS Логис...
7,ИС 1С: TMS настройка приложения - в проде;http...
8,Калькулятор FTL - MVP (но еще не обновлена);ht...
9,Карточка контрагента. Создание адреса доставки...


In [141]:
links[['Title', 'URL']] = links[0].str.split(';', n=1, expand=True)
links = remove_punctuation(links, ['Title'])
df_ftl = remove_punctuation(df_ftl, ['Ans', 'Quest'])
df_ftl = df_ftl.rename(columns={'Query': 'Quest', 'Answer': 'Ans'})
links['Sovpad'] = links['Title'].apply(
    lambda x: (process.extractOne(x, df_ftl['Quest'], score_cutoff=75) or (None,))[0]
)

best_match_to_link = dict(zip(links['Sovpad'], links['URL']))

def find_link(query):
    for best_match, link in best_match_to_link.items():
        if best_match is not None and query.startswith(best_match):
            return link
    return None

df_ftl['URL'] = df_ftl['Quest'].apply(find_link)

df_ftl['Department'] = 'FTL'
df_ftl.to_csv('df_drk.csv', index=False)

df_ftl

,Quest,Ans,URL,Department
0,Редактирование географических зон,Открыть обработку «Редактирование географичес...,http://confluence/pages/viewpage.action?pageId...,FTL
1,Услуги по страхованию груза и срока при перево...,EPIC-11060-Получение подробных данных проблемы...,http://confluence/pages/viewpage.action?pageId...,FTL
2,Отчет по КБ,3.1.Период– это дата подачи ТС под загрузку по...,http://confluence/pages/viewpage.action?pageId...,FTL
3,Отчет по агентским вознаграждениям - MVP,"По каждому рейсу, где клиент-заказчик был прив...",http://confluence/pages/viewpage.action?pageId...,FTL
4,Калькулятор FTL - MVP (но еще не обновлена),https://evgenia-petrova.atlassian.net/browse/F...,http://confluence/pages/viewpage.action?pageId...,FTL
5,Комплекты документов - MVP,EPIC-10380-Получение подробных данных проблемы...,http://confluence/pages/viewpage.action?pageId...,FTL
6,Отчет по рейсам - MVP,EPIC-10889-Получение подробных данных проблемы...,http://confluence/pages/viewpage.action?pageId...,FTL
7,Обработка для сотрудников бухгалтерии - MVP,"Обработка находится в разделе ""Сервисы"" Досту...",http://confluence/pages/viewpage.action?pageId...,FTL
8,ИС 1С_ TMS настройка приложения,Целевая группа – пользователи и администратор ...,None,FTL
9,Профили групп доступа,Группа доступа Группа доступа Разрешенные дейс...,http://confluence/pages/viewpage.action?pageId...,FTL


## OTP

In [146]:
directory_otp = directory + 'ОТР'

paragraphs_dataframes = [
    pd.DataFrame(
        [[
            os.path.splitext(filename)[0],
            ' '.join(
                tag.get_text(strip=True)
                for tag in (lambda soup: (
                    [t.decompose() for t in soup.find_all(['s', 'strike', 'del'])] or soup,
                    soup.find_all(['p', 'td'])
                )[1])(BeautifulSoup(open(os.path.join(directory_otp, filename), encoding='utf-8').read(), 'html.parser'))
            )
        ]],
        columns=['Query', 'Answer']
    )
    for filename in os.listdir(directory_otp)
    if os.path.isfile(os.path.join(directory_otp, filename)) and filename.endswith('.html')
]

df_otp = pd.concat(paragraphs_dataframes, ignore_index=True)
df_otp

,Query,Answer
0,ОТР (DPEGAS-36205),Описание технической реализации DPEGAS-36205-П...
1,ОТР (DPEGAS-36085),Описание технической реализации DPEGAS-36085-П...
2,ОТР (DPEGAS-36120),Описание технической реализации DPEGAS-36120...
3,ОТР (DPEGAS-36079),Описание технической реализации DPEGAS-36079...
4,ОТР (DPEGAS-36210),Описание технической реализации DPEGAS-36210-П...
...,...,...
210,ОТР (DPEGAS-29892),Описание технической реализации DPEGAS-29892...
211,ОТР (DPEGAS-22372),Описание технической реализации DPEGAS-22372-П...
212,ОТР (DPEGAS-30134),Описание технической реализации DPEGAS-30134-...
213,ОТР (DPEGAS-22351),Описание технической реализации DPEGAS-22351-П...


In [147]:
fixtext1 = 'ОЦЕНОЧНАЯ ЛИЦЕНЗИЯВам нравится Confluence? Пожалуйста, рассмотритедля приобретения егосегодня.'
fixtext2 = 'СТАТУС'

df_otp['Answer'] = df_otp['Answer'].str.replace(fixtext1, '', regex=False)
df_otp['Answer'] = df_otp['Answer'].str.replace(fixtext2, '', regex=False)

df_otp

,Query,Answer
0,ОТР (DPEGAS-36205),Описание технической реализации DPEGAS-36205-П...
1,ОТР (DPEGAS-36085),Описание технической реализации DPEGAS-36085-П...
2,ОТР (DPEGAS-36120),Описание технической реализации DPEGAS-36120...
3,ОТР (DPEGAS-36079),Описание технической реализации DPEGAS-36079...
4,ОТР (DPEGAS-36210),Описание технической реализации DPEGAS-36210-П...
...,...,...
210,ОТР (DPEGAS-29892),Описание технической реализации DPEGAS-29892...
211,ОТР (DPEGAS-22372),Описание технической реализации DPEGAS-22372-П...
212,ОТР (DPEGAS-30134),Описание технической реализации DPEGAS-30134-...
213,ОТР (DPEGAS-22351),Описание технической реализации DPEGAS-22351-П...


In [148]:
links = pd.read_csv(directory + 'OTR.txt', sep=';')
links

,name,value
0,ОТР (DARCH-1059),http://confluence/pages/viewpage.action?pageId...
1,ОТР (DBUH-1106),http://confluence/pages/viewpage.action?pageId...
2,ОТР (DBUH-1109),http://confluence/pages/viewpage.action?pageId...
3,ОТР (DBUH-1117),http://confluence/pages/viewpage.action?pageId...
4,ОТР (DBUH-1125),http://confluence/pages/viewpage.action?pageId...
...,...,...
210,ОТР (RND-910),http://confluence/pages/viewpage.action?pageId...
211,ОТР (RND-933),http://confluence/pages/viewpage.action?pageId...
212,ОТР (RND-937),http://confluence/pages/viewpage.action?pageId...
213,ОТР (RND-939),http://confluence/pages/viewpage.action?pageId...


In [149]:
links = links.rename(columns={'name': 'Title', 'value': 'URL'})

links = remove_punctuation(links, ['Title'])
df_otp = remove_punctuation(df_otp, ['Ans', 'Quest'])
df_otp = df_otp.rename(columns={'Query': 'Quest', 'Answer': 'Ans'})
links['Sovpad'] = links['Title'].apply(
    lambda x: (process.extractOne(x, df_otp['Quest'], score_cutoff=75) or (None,))[0]
)

best_match_to_link = dict(zip(links['Sovpad'], links['URL']))

def find_link(query):
    for best_match, link in best_match_to_link.items():
        if best_match is not None and query.startswith(best_match):
            return link
    return None

df_otp['URL'] = df_otp['Quest'].apply(find_link)

df_otp['Department'] = 'OTP'
df_otp.to_csv('df_otp.csv', index=False)

df_otp

,Quest,Ans,URL,Department
0,ОТР (DPEGAS-36205),Описание технической реализации DPEGAS-36205-П...,http://confluence/pages/viewpage.action?pageId...,OTP
1,ОТР (DPEGAS-36085),Описание технической реализации DPEGAS-36085-П...,http://confluence/pages/viewpage.action?pageId...,OTP
2,ОТР (DPEGAS-36120),Описание технической реализации DPEGAS-36120...,http://confluence/pages/viewpage.action?pageId...,OTP
3,ОТР (DPEGAS-36079),Описание технической реализации DPEGAS-36079...,http://confluence/pages/viewpage.action?pageId...,OTP
4,ОТР (DPEGAS-36210),Описание технической реализации DPEGAS-36210-П...,http://confluence/pages/viewpage.action?pageId...,OTP
...,...,...,...,...
210,ОТР (DPEGAS-29892),Описание технической реализации DPEGAS-29892...,http://confluence/pages/viewpage.action?pageId...,OTP
211,ОТР (DPEGAS-22372),Описание технической реализации DPEGAS-22372-П...,http://confluence/pages/viewpage.action?pageId...,OTP
212,ОТР (DPEGAS-30134),Описание технической реализации DPEGAS-30134-...,http://confluence/pages/viewpage.action?pageId...,OTP
213,ОТР (DPEGAS-22351),Описание технической реализации DPEGAS-22351-П...,http://confluence/pages/viewpage.action?pageId...,OTP
